## Bayesian methods of hyperparameter optimization

In [ ]:
#USE ANACONDA3 ENV!

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
#! pip install bayesian-optimization
#Running a cell with an exclamation point in the beginning of the command will run it as a shell command —
#please do this to install this module from our notebook in the cell below

#HAIN?!?! install it from YOUR notebook??? what'll that do??? why not install it ourSELVES?!?
##################################################################################################################

In [3]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
################################################
import numpy as np
import pandas as pd
import lightgbm
################################################
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [2]:
#worked!
# conda install -c conda-forge catboost

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /Users/deens/opt/anaconda3

  added / updated specs:
    - catboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    atk-1.0-2.36.0             |       he69c4ee_4         359 KB  conda-forge
    cairo-1.16.0               |    h9e0e54b_1010         1.4 MB  conda-forge
    catboost-1.1.1             |   py39h6e9494a_1        19.9 MB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0          94 KB  conda-forge
    font-ttf-source-code-pro-2.038|       h77eed37_0         684 KB  conda-forge
    font-ttf-ubuntu-0.83       |       hab24e00_

In [11]:
#worked
# pip install -U colorama

  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5
Note: you may need to restart the kernel to use updated packages.


In [3]:
# conda install -c conda-forge scikit-learn
# worked!
# used anaconda3 env! not base, not tf_m1! those didn't work

Solving environment: done

## Package Plan ##

  environment location: /Users/deens/opt/anaconda3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       h033912b_0         142 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    conda-22.11.1              |   py39h6e9494a_1         874 KB  conda-forge
    joblib-1.2.0               |     pyhd8ed1ab_0         205 KB  conda-forge
    openssl-3.0.7              |       hfd90126_1         2.2 MB  conda-forge
    pluggy-1.0.0               |     pyhd8ed1ab_5          16 KB  conda-forge
    ruamel.yaml-0.17.21        |   py39ha30fb19_2         173 KB  conda-forge
    ruamel.yaml.clib-0.2.7     |   py39ha30fb19_1         118 KB  conda-forge
    scikit-learn-1.1.3         |   py39hdbdcc14_1         7.7 MB  conda-forge
    -----

In [5]:
#worked!
# conda install -c conda-forge lightgbm

Retrieving notices: ...working... /Users/deens/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repo.anaconda.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/deens/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'conda.anaconda.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/deens/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repo.anaconda.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warning

In [7]:
#worked!
# conda install -c conda-forge bayesian-optimization

Solving environment: done

## Package Plan ##

  environment location: /Users/deens/opt/anaconda3

  added / updated specs:
    - bayesian-optimization


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bayesian-optimization-1.4.2|     pyhd8ed1ab_0          20 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          20 KB

The following NEW packages will be INSTALLED:

  bayesian-optimiza~ conda-forge/noarch::bayesian-optimization-1.4.2-pyhd8ed1ab_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
os.listdir()

#'os' is operating system, checked, but why is that the thing for where these files are? this is just this folder
#/location, aka like 'cd' or just '/' auto?

['flight_delays_test.csv.zip',
 '.DS_Store',
 'flight_delays_train.csv.zip',
 'Bayesian Optimization Case Study - WORKING COPY.ipynb',
 '.ipynb_checkpoints',
 'Bayesian Optimization Case Study - Original Copy.ipynb']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [4]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [5]:
optimizer = BayesianOptimization(simple_func,{'a': (1, 3),'b': (4, 7)})
#so it goes, BO(func,bounds_dict)
#REPRESENTING THE ONE-TWO-STEPS! ;P

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

In [ ]:
#okay he said MAXIMUM - so maybe we do inverse
#hmmm okay don't remember if we played w/ these 2 settings?
#but this does sound like the stuff we know about - like init_points must be the number of 'pre'/exploration/
#CALIBRATION steps!!!!!!
#i remember we saw that, but don't know if we explicitly programmed or accepted default
#and then n_iter is after that, once you hone in on the areas to concentrate on/cluster(s/range) of highest density,
#and that i think i remember we did specify like 10,000? or that was the default again
#have to go back n check

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [16]:
optimizer.maximize(3,2)
#well if you look at the docx, these should be
#init_points & n_iter...
#so 3 initial calibration steps and 2 actual / production lol / 'forreal' steps
#OHHHHHHH >>> SO IS THAT WHY THERE'S 5 TOTAL STEPS HERE?!?!?
#but i didn't think they included/combined the calibration & production ones
#together?!?! but also - why does it need ANY calibration
#if you can literally just go straight to the max values of each??
#and what is it even learning from the calibration lol
#if so few points?? oh, well, remem it's a RANGE!
#so can be ANY INFINITE NUMBER OF DECIMALS BETWEEN!
#but like, if you look at it - like how did it even
#conclude on what range to focus on / flesh out
#in calibration?! like how could it do that in just 2 steps?
#spreads out? and how did it suddenly figure out to try
#exactly the max values lol?
####################################################

#okay so "find the best values to maximize the target value for a & b given the inputs of 3 & 2"...
#so are 3 & 2 supposed to be a & b, resp?
#the function is a + b, so if ^ so, then 3 + 2 = 5
#OH, and remem, we specified the range of a & b above! 1->3 & 4->7, resp
#thus... i guess it can't be that 3,2 reps a,b? cuz 2 is out of range for b....
#okay, so, if we look below, the winner is 'target = 10, from a=3 + b=7',....
#okay so - that's LITERALLY simply maximizing the function given the ranges, cuz 3 & 7 are RANGE TOPPERS resp!!!!
#but then.... WHAT THE HECK ARE 3 & 2?!?!?!?!?!!?
#like it's saying find the values that maximize the target value....
#so how do we know what the target value is, again - if it's not 3,2?
#if 10 is the TARGET value, and it's just equal to using the MAXIMUMS of the params ranges, then
#'maximizing' the target value literally means just making it the maximum possible lolll
#but i guess i'm confused on what makes it a TARGET value if it can *change*? unless 'target value' is simply
#terminology for the result of the function?
#ohhhhhhh - yeah that could be it, like just like how 'target feature' means *y*!!!!!!! aka the thing
#you're tryna predict! here, it's not predictions but the y of the function we choose is the thing
#we're using to indirectly (but proportionally?) represent our METRIC like via *PROBABILITY DENSITY*
#and we get the MAX of that, aka MAX PROBABILITY DENSITY, and take the PARAMETER VALUE THAT GIVES THAT MAX!!!!!
###############################################################################################

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 8.467     | 2.727     | 5.741     |
| 2         | 5.952     | 1.934     | 4.018     |
| 3         | 9.42      | 2.69      | 6.73      |
| 4         | 8.0       | 1.0       | 7.0       |
| 5         | 10.0      | 3.0       | 7.0       |


In [ ]:
#oh, my output is different than theirs?? is it cuz of RANDOMNESS?? no random_state /_seed option??
#and why are there *TWO* highlighted?? theirs only had 1 - but at least the 'max' value/row is the same
############################################################################################################

Great, now let's print the best parameters and the associated maximized target.

In [17]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

In [ ]:
#okay so yeah aH at least this is the same which is what matters

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [18]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [19]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [ ]:
#so based on month, day of month, day of WEEK, time of day, carrier, origin, destination & distance
#then predictor/target feature is BINARY yes/no for whether delayed 15 mins or not (AT LEAST 15 mins, obv,
#i'm assuming). So note, basically the definition for 'delay' is that it was AT LEAST 15 minutes! if less than that,
#then grace period lol, off the hook

**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [20]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


In [ ]:
#100,THOUSAND ENTRIES!!!!!

In [ ]:
#ohhh right, so automatically ONLY NUMERICAL FEATURES ARE INCLUDED!!!!

Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

In [ ]:
#hmm, is it appropriate for us to use TIME tho, since it's CIRCULAR? that's interesting right,
#almost like its OWN SEPARATE CATEGORY! from discrete & continuous (and not 'discrete style continuous either lol)
#cuz like 2359 (11:59PM) is basically the SAME as 0000 (12AM MIDNIGHT!)... but they're MAXIMALLY SEPARATED
#ON THIS SCALE!!!!!!!! SO HOW DOES THAT WORK?!?!
######################################################################################################

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [22]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


In [ ]:
#nice no at least APPARENT/LITERAL BLANKS!!!!! doesn't guarantee no MISSINGS or BALONEYS/BOGEYS!!!!!/BOGUESS! :P

In [21]:
train_df.dep_delayed_15min.unique()

array(['N', 'Y'], dtype=object)

In [42]:
#train_df = train_df[train_df.DepTime <= 2400].copy() #>> they don't say anything about this, but is this
#to filter bogus values? OH! yeah if we look @/to the describe table, we see the max time *IS* >2400!!!!!
#SO THERE MAY BE OTHERS!!!!
#########################################################################################################
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
#SO REMEM! .MAP() IS A SERIOUS TOOL! YOU HAVE TO USE IT VERY CAREFULLY/BE VERY SURE!
#compared to .replace(manual_mapping_dict), like we usually do, .map() will RENDER ANYTHING THAT'S *NOT*
#A MATCH AKA *NOT* CONTAINED IN THE DICT AS A *NAN*!!!!!!!!!
#but we can check w/ a .unique, esp w/ binaries it's very easy
#and maybe it's that the only non-specified values ARE nans, so those won't be affected lol, but we shoulda
#started w/ confirming there aren't any of those/removing any if there are!
#checked above^ and we're good!

In [ ]:
#ohhhh okay, sA, just realized this is different than what we usually do cuz
#we're working on / treating / cleaning & doing EDA *ONLY* on the TRAINING
#portion/split?!?!?! WHY'S THAT?!? why didn't we START with the FULL??
#is this like SUPER 'data leakage' / cybersecurity paranoia?! lol
########################################################################

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [33]:
#DON'T KNOW! but gives us mapping we use later
def label_enc(df_column): #hain?
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400): #hain?
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400): #hain?
    value *= 2 * np.pi / period 
    return np.cos(value)

#'MANUFACTURED FEATURES' AKA *FEATURE ENGINEERING* INDEED!!!!!
def feature_eng(df):
    #THESE GIVE US *ADDITIONAL GROUPINGS* FOR FLIGHTS SO THAT WE CAN SEE WHAT OTHER FLIGHTS WERE THAT SAME COMBO!
    df['flight'] = df['Origin']+df['Dest'] #i guess if there are REPEAT routes in here then we can see if there are any patterns/consistencies w/ those! note that ORDER MATTERS!!!! like CHI-SEA is diff than SEA-CHI!!!!! MAKES SENSE THO!!!!!
    df['flightUC'] = df['flight']+df['UniqueCarrier'] #this one builds off what we just made!
    df['DestUC'] = df['Dest']+df['UniqueCarrier'] #so like MDW-SOUTHWEST
    df['OriginUC'] = df['Origin']+df['UniqueCarrier'] #also could be MDW-SOUTHWEST! but since these are DIFF columns, they would be totally separate! like it wouldn't make sense for the origin and destination city to be the same!

    #these ones just clean up the formatting, like removing the weird 'c-' prefix from these date monikers
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')

    #puts a decimal between the first two//hour numbers & second two//minute numbers of the military time stamp,
    #i guess to make easier to read? kinda serving like purpose of a 'colon:' but RETAINS its NUMERICAL identity!
    #which again i'm skeptical about
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')

    #these ones create categories / labels / groups! but rather than CUTTING which REPLACES specifics/numbers
    #w/ broader general categories, this is in ADDITION TO... But i guess we could achieve the same thing
    #w/ cutting if we just created a COPIED COLUMN?!?!
    ########################################################################
    #labels for time of month
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    #labels for time of day
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    #labels for time of YEAR!
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')

    #labels for Weekend vs. Weekday... though, it says 'Holiday'... but that's misleading, though a GOOD
    #idea to have a THIRD if possible for ACTUAL like federal US holidays!!!
    ########################################################################
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)

    #These are like counts/metrics pertaining to one piece of that row/record
    #wait these all do *.TRANSFORM* - isn't that only for MODELS that have been FIT?!
    ########################################################################
    #we can make sense of everything OUTSIDE of the .transform(), aka the GROUPBY, but don't understand the .transform()...
    #so i'll at least comment on the groupbys counts
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count') #dest airport & month count
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count') #orig airport & month count
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count') #just dest airport count
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count') #TOTAL number of origins... couldn't we just do like len(df.col.unique())? or df.col.unique().count()??
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count') #TOTAL number of destinations each carrier/airline had
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count') #number of destinations that airline had each month
    
    #OHHHHHHHHHHHHHHHHHHHHHHHHHHHH!
    #okay now aH i think i get it for what .transform is doing
    #so, this is something i've done in my own excel analytics gymnastics acrobatics lol
    #it's basically putting some summary value for some grouping related to that row, for *EVERY* grouping in that row!!!
    #so, we start off w/ each of these as making a new column right
    #so the purpose of that column is to have the value of the grouping that pertains to THAT ROW!!!!
    #so EVERY member/row belonging to a grouping has the *SAME* exact value for that column!!!!!!!!!!
    #so like, let's pick a simple one - airport_origin_count - so that's ONLY grouped on every ORIGIN,
    #thus, remem w/ each row being a *FLIGHT*, ALL the flights of the SAME origin will show the SAME NUMBER/VALUE
    #FOR THAT COLUMN OF THE TOTAL NUMBER OF (SUM) FLIGHTS WHERE THAT ORIGIN AIRPORT WAS THE ORIGIN AIRPORT!
    #SO AKA HOW MANY FLIGHTS IN THIS DATAFRAME ARE FROM THAT ORIGIN!!!!!!
    #SO SAY/TAKE CHICAGO MIDWAY (MDW) - SAY 100 OF THE FLIGHTS IN THIS DATAFRAME ARE MIDWAY ORIGIN FLIGHTS;
    #THEN, FOR THIS 'AIRPORT_ORIGIN_COUNT' FIELD, FOR *EVERY* MIDWAY FLIGHT, *IT'S GONNA SAY/BE *100*!!!!!!
    ####################################################################################################

    
    #STILL FIGURING WHAT THIS IS ALL ABOUT
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)


    return df.drop('DepTime', axis=1)

In [31]:
train_df.groupby(['UniqueCarrier', 'Month'])['Dest'].count()

UniqueCarrier  Month
AA             c-1      797
               c-10     770
               c-11     756
               c-12     785
               c-2      708
                       ... 
YV             c-5      170
               c-6      190
               c-7      198
               c-8      194
               c-9      179
Name: Dest, Length: 261, dtype: int64

In [26]:
#okay so if we take this as an example:
#train_df.groupby(['Dest', 'Month'])['Dest'].transform('count')

#well, let's take out the transform for now and see

#okay so this does what we expect aH - groups each destination airport & month combo
#and simply counts how many times each combo comes up

train_df.groupby(['Dest', 'Month'])['Dest'].count()

Dest  Month
ABE   c-1      7
      c-10     3
      c-11     7
      c-12     4
      c-2      8
              ..
YUM   c-5      5
      c-6      3
      c-7      3
      c-8      2
      c-9      2
Name: Dest, Length: 3037, dtype: int64

In [24]:
#okay so then... what the heck is .transform doing?
train_df.groupby(['Dest', 'Month'])['Dest'].transform('count')

0        373
1        168
2        104
3         46
4         21
        ... 
99995      2
99996      4
99997    266
99998      2
99999     68
Name: Dest, Length: 100000, dtype: int64

In [ ]:
##################################################
#hmmm - yeah not sure... it's got one number for every ROW in the df... but what
#would that number represent? but also, it's a transform on the GROUPBY/pivot...
#which ISN'T all the diff original indiv rows.... So not sure how it's coming up w/ that??
##################################################

In [34]:
# help(transform)

Concatenate the training and testing dataframes.


In [ ]:
#lol reverse of how we usually do it... but kinda like serious segregation

In [36]:
#DROP THE THING WE'RE TRYNA PREDICT! like making the groom leave the room for the bride lollll
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

#hmm okay, speaking of groom, w/ this now we're gonna 'groom' the FULL dataset where only the
#training portion was treated - .... OHHHHHHH! okay, so no - not really, wasn't 'treated' - not the X_FEATURES matrix
#anyway - JUST the y's!!!!!! and that makes sense cuz we need the y's for TRAINING but *NOT* to be included in TEST!!!
#until the END!

Apply the earlier defined feature engineering functions to the full dataframe.

In [37]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [38]:
#so... just splitting it BACK to train & test, except this time WITHOUT the y, and now feature_engineered features
#just using the SHAPE of original train & test
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [39]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

In [ ]:
#grape. GradientBoosting was the one thing i didn't really learn

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [40]:
#dang so we gotta put in ALL THESE?! and NONE OF 'EM HAVE DEFAULTS?!?!
########################################################################
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [43]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=444, min_child_samples=6077 will be ignored. Current value: min_data_in_leaf=444
| 1         | 0.7193    | 0.002822  | 0.03023   | 49.67     | 6.078e+03 | 444.6     | 678.6     |
| 2         | 0.7194    | 0.04816   | 0.01211   | 9.398     | 6.357e+03 | 308.9     | 1.189e+03 |
| 3         | 0.7437    | 0.04154   | 0.01364   | 9.394     | 5.024e+03 | 1.975e+03 | 2.909e+03 |
| 4         | 0.7436    | 0.04682   | 0.01212   | 31.43     | 3.594e+03 | 1.994e+03 | 3.903e+03 |
| 5         | 0.728     | 0.00967   | 0.01261   | 60.63     | 2.466e+03 | 878.2     | 971.1     |
| 6         | 0.7437    | 0.04369   | 0.04999   | 52.54     | 6.162e+03 | 1.961e+03 | 3.999e+03 |
| 7         | 0.7436    | 0.01034   | 0.005276  | 40.56     | 7.236e+03 | 1.479e+03 | 2.31

In [ ]:
#wait.... dannnggggg it! whattha higgy siggy - so ok i guess my theory WASN'T right then?! or part of it was...
#so here we did n_iter=10, and indeed it did 10 runs, but it didn't follow last thing of 3 + 2 = 5?!?!??!?!?!
#so explain that to me......
#but again, at the same time, that's what i thought that the only the n_iters are the ones that display/'production' runs
#that show, and not the calibrators aka INITIAL 'points'/runs... but then again, how/why did we get 5 total before?!
#i guess double check what the 3,2 were! why they weren't explicit!?!
####################################################################################################################

In [ ]:
#ohhh, so if you watch it in real-time come up w/ this list / go thru these iterations,
#it makes purple the CURRENT max/best up till then?? but then it shoulda made the VERY FIRST purp then
#no? maybe it excludes that. AHHHH, yeah, so i think that's right! that adds up! is consistent with what we see!
#cuz if you look at the TARGET column, which is what we're after the max of, like back in our run
#in the beginning when we were confused on why there were 2 purple rows, it's cuz TWICE the current run's target
#was the NEW HIGH! and yeah, the very last one ended up being the HIGHEST and MAX HIGH POSSIBLE?!
#and if you look at their pre-loaded version, the 4th was the overall max, and it worked out that yeah - that's
#correct, NO OTHER ROW SHOULD'VE LIT UP/BEEN HIGHLIGHTED CUZ NO ONE EVER DETHRONED THE CURRENT MAX (lol like a name!
# like, who's the MAX?!)/ KING'S COURT! 7.8, 7.1, 7.4 [10.0], 9.4!!!!!!!
#############################################################################################################

In [ ]:
#SO WHAT WAS THAT WHOLE *GIF* THING ABOUT RELATIVE TO THIS?!?

In [ ]:
#what's the warning about?:
#[Warning] min_data_in_leaf is set=444, min_child_samples=6077 will be ignored.
#Current value: min_data_in_leaf=444

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [44]:
lgbBO.max

{'target': 0.743677184606366,
 'params': {'lambda_l1': 0.04368878406260319,
  'lambda_l2': 0.04998807104235295,
  'max_depth': 52.535726432546326,
  'min_child_samples': 6161.725956471824,
  'min_data_in_leaf': 1960.867045207072,
  'num_leaves': 3998.716330891756}}

Review the process at each step by using the '.res[0]' function.

In [45]:
lgbBO.res[0]

{'target': 0.7192540779609389,
 'params': {'lambda_l1': 0.002822101387856496,
  'lambda_l2': 0.030229580059482154,
  'max_depth': 49.66714825748048,
  'min_child_samples': 6077.71394513601,
  'min_data_in_leaf': 444.62104547614024,
  'num_leaves': 678.6063438891952}}

In [ ]:
#ohh okay, so lgbBO ('light gradient boosting bayesian optimizer'?) MAX is the MAX target run/RES,
#aka [RES] is AN INDIVIDUAL RUN! so this one, [0] is simply the first one above!